In [1]:
## Importing required libraries
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from time import time
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
import matplotlib
matplotlib.rcParams['figure.figsize'] = [20, 12]

In [2]:
def get_match_label(match):
    ''' Derives a label for a given match. '''
    
    #Define variables
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']

    #Identify match label  
    if home_goals > away_goals:
        return 'Win'
    if home_goals == away_goals:
        return 'Draw'
    if home_goals < away_goals:
        return 'Defeat'

In [3]:
def get_last_matches(matches, date, team, x = 10):
    ''' Get the last x matches of a given team. '''
    
    #Filter team matches from matches
    team_matches = matches[(matches['home_team_api_id'] == team) | (matches['away_team_api_id'] == team)]
                           
    #Filter x last matches from team matches
    last_matches = team_matches[team_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:x,:]
    
    #Return last matches
    return last_matches

In [4]:
def get_last_matches_against_eachother(matches, date, home_team, away_team, x = 10):
    ''' Get the last x matches of two given teams. '''
    
    #Find matches of both teams
    home_matches = matches[(matches['home_team_api_id'] == home_team) & (matches['away_team_api_id'] == away_team)]    
    away_matches = matches[(matches['home_team_api_id'] == away_team) & (matches['away_team_api_id'] == home_team)]  
    total_matches = pd.concat([home_matches, away_matches])
    
    #Get last x matches
    try:    
        last_matches = total_matches[total_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:x,:]
    except:
        last_matches = total_matches[total_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:total_matches.shape[0],:]
        
        #Check for error in data
        if(last_matches.shape[0] > x):
            print("Error in obtaining matches")
            
    #Return data
    return last_matches

In [5]:
def convert_odds_to_prob(match_odds):
    ''' Converts bookkeeper odds to probabilities. '''
    
    #Define variables
    match_id = match_odds.loc[:,'match_api_id']
    bookkeeper = match_odds.loc[:,'bookkeeper']    
    win_odd = match_odds.loc[:,'Win']
    draw_odd = match_odds.loc[:,'Draw']
    loss_odd = match_odds.loc[:,'Defeat']
    
    #Converts odds to prob
    win_prob = 1 / win_odd
    draw_prob = 1 / draw_odd
    loss_prob = 1 / loss_odd
    
    total_prob = win_prob + draw_prob + loss_prob
    
    probs = pd.DataFrame()
    
    #Define output format and scale probs by sum over all probs
    probs.loc[:,'match_api_id'] = match_id
    probs.loc[:,'bookkeeper'] = bookkeeper
    probs.loc[:,'Win'] = win_prob / total_prob
    probs.loc[:,'Draw'] = draw_prob / total_prob
    probs.loc[:,'Defeat'] = loss_prob / total_prob
    
    #Return probs and meta data
    return probs

### Doing things now...

In [6]:
## Fetching data
#Connecting to database
path = "soccer_database.sqlite"  #Insert path here
conn = sqlite3.connect(path)

#Defining the number of jobs to be run in parallel during grid search
n_jobs = 2 #Insert number of parallel jobs here

#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match where country_id = 1729;", conn)

match_data.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,10.00,1.28,5.5,12.00,1.30,4.75,10.0,1.29,4.50,11.00
1,1730,1729,1729,2008/2009,1,2008-08-16 00:00:00,489043,9825,8659,1,...,12.00,1.25,6.0,13.00,1.22,5.50,13.0,1.22,5.00,13.00
2,1731,1729,1729,2008/2009,1,2008-08-16 00:00:00,489044,8472,8650,0,...,1.73,5.50,3.8,1.65,5.00,3.40,1.7,4.50,3.40,1.73
3,1732,1729,1729,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,...,3.75,1.90,3.5,4.35,1.91,3.25,4.0,1.91,3.25,3.80
4,1733,1729,1729,2008/2009,1,2008-08-17 00:00:00,489046,10252,8456,4,...,3.75,1.90,3.5,4.35,1.91,3.25,4.0,1.91,3.30,3.75


In [7]:
match_data.groupby('season').size()

season
2008/2009    380
2009/2010    380
2010/2011    380
2011/2012    380
2012/2013    380
2013/2014    380
2014/2015    380
2015/2016    380
2016/2017    380
2017/2018    380
dtype: int64

In [8]:
match_data[match_data.season == '2017/2018'].head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
3040,55030,1729,51959,2017/2018,32,2018-04-07 00:00:00,2523055,8455,8586,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3041,55031,1729,51959,2017/2018,32,2018-04-07 00:00:00,2523057,8668,8456,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3042,55032,1729,51959,2017/2018,32,2018-03-31 00:00:00,2523056,9826,8650,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3043,55033,1729,51959,2017/2018,32,2018-04-07 00:00:00,2523053,9825,10194,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3044,55034,1729,51959,2017/2018,32,2018-04-07 00:00:00,2522762,8654,8466,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
## Man U 10260
match_data[match_data.home_team_api_id == 10260].groupby('season').size()

season
2008/2009    19
2009/2010    19
2010/2011    19
2011/2012    19
2012/2013    19
2013/2014    19
2014/2015    19
2015/2016    19
2016/2017    19
2017/2018    19
dtype: int64

In [26]:
match_data[match_data.season == '2016/2017'].shape

(380, 115)

In [7]:
with_team_names = match_data.merge(
    team_data, 
    how='inner', 
    left_on=['home_team_api_id'], 
    right_on=['team_api_id'], 
    suffixes=('', '_home')
).merge(
    team_data, 
    how='inner', 
    left_on=['away_team_api_id'], 
    right_on=['team_api_id'], 
    suffixes=('', '_away')
)
with_team_names.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,id_home,team_api_id,team_fifa_api_id,team_long_name,team_short_name,id_away,team_api_id_away,team_fifa_api_id_away,team_long_name_away,team_short_name_away
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,3457,10260,11.0,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW
1,2494,1729,1729,2010/2011,1,2010-08-16 00:00:00,839802,10260,10261,3,...,3457,10260,11.0,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW
2,2913,1729,1729,2011/2012,13,2011-11-26 00:00:00,1024845,10260,10261,1,...,3457,10260,11.0,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW
3,3353,1729,1729,2012/2013,19,2012-12-26 00:00:00,1229300,10260,10261,4,...,3457,10260,11.0,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW
4,3693,1729,1729,2013/2014,15,2013-12-07 00:00:00,1474518,10260,10261,0,...,3457,10260,11.0,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW


In [8]:
import operator
from xml.etree import ElementTree as ET

def get_posession_things(data):
    last_value = 50
    try:
        tree = ET.fromstring(data)
        results = tree.findall('value')
        results_dict = dict((result.find('elapsed').text, result.find('homepos').text) for result in results)
        last_value = [value for key, value in sorted(results_dict.items(), key=operator.itemgetter(1), reverse=True)][0]
    except:
        pass
    
    return last_value

In [9]:
with_team_names.loc[:, 'home_possession'] = with_team_names.possession.apply(lambda x: get_posession_things(x))

In [10]:
with_team_names.loc[:, 'total_prob'] = 1./with_team_names.B365H + 1./with_team_names.B365D + 1./with_team_names.B365A
with_team_names.loc[:, 'pct_win_prob'] = (1./ with_team_names.B365H) / with_team_names.total_prob
with_team_names.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,team_long_name,team_short_name,id_away,team_api_id_away,team_fifa_api_id_away,team_long_name_away,team_short_name_away,home_possession,total_prob,pct_win_prob
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW,56,1.047921,0.739744
1,2494,1729,1729,2010/2011,1,2010-08-16 00:00:00,839802,10260,10261,3,...,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW,64,1.048485,0.763006
2,2913,1729,1729,2011/2012,13,2011-11-26 00:00:00,1024845,10260,10261,1,...,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW,70,1.054155,0.677591
3,3353,1729,1729,2012/2013,19,2012-12-26 00:00:00,1229300,10260,10261,4,...,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW,57,1.050441,0.780312
4,3693,1729,1729,2013/2014,15,2013-12-07 00:00:00,1474518,10260,10261,0,...,Manchester United,MUN,3458,10261,13.0,Newcastle United,NEW,48,1.027399,0.636165


## Work out how good each team is

In [11]:
HISTORY = 10
k_factor = 32
elo_width = 400.

def calculate_new_elos(rating_a, rating_b, score_a, k_factor, elo_width):
    """Calculates and returns the new Elo ratings for two players.
    score_a is 1 for a win by player A, 0 for a loss by player A, or 0.5 for a draw.
    """

    e_a = expected_result(rating_a, rating_b, elo_width)
    e_b = 1. - e_a
    new_rating_a = rating_a + k_factor * (score_a - e_a)
    score_b = 1. - score_a
    new_rating_b = rating_b + k_factor * (score_b - e_b)
    return new_rating_a, new_rating_b

def expected_result(elo_a, elo_b, elo_width):
    """
    https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
    """
    expect_a = 1.0/(1+10**((elo_b - elo_a)/elo_width))
    return expect_a

def update_end_of_season(elos):
    """Regression towards the mean
    
    Following 538 nfl methods
    https://fivethirtyeight.com/datalab/nfl-elo-ratings-are-back/
    """
    diff_from_mean = elos - np.mean(elos)
    elos -= diff_from_mean/3
    return elos

In [39]:
team_elo = with_team_names.team_long_name.drop_duplicates().reset_index()
team_elo.loc[:, 'elo'] = 1000
team_elo = team_elo[['team_long_name', 'elo']]
with_team_names.loc[:, 'expected'] = 0.5
team_elo.head()

,team_long_name,elo
0,Manchester United,1000
1,Arsenal,1000
2,Sunderland,1000
3,West Ham United,1000
4,Aston Villa,1000


In [14]:
def calculate_result(row):
    if row.home_team_goal > row.away_team_goal:
        return 1
    elif row.home_team_goal == row.away_team_goal:
        return 0.5
    else:
        return 0

In [15]:
def create_predictions(team_elo, col_name, k_factor, elo_width):
    current_season = '2008/2009'
    for index, row in with_team_names.sort_values(by='date').iterrows():
        new_season = row.season
        if new_season != current_season:
            new_elos = update_end_of_season(team_elo.elo.values)
            team_elo.loc[:, 'elo'] = new_elos
            current_season = new_season

        home = row.team_long_name
        away = row.team_long_name_away
        home_elo = team_elo[team_elo.team_long_name == home].elo.values[0]
        away_elo = team_elo[team_elo.team_long_name == away].elo.values[0]
        expected_results = expected_result(home_elo, away_elo, elo_width)
        result = calculate_result(row)
        with_team_names.loc[index, col_name] = expected_results
        new_home_elo, new_away_elo = calculate_new_elos(home_elo, away_elo, result, k_factor, elo_width)
        team_elo.loc[team_elo.team_long_name == home, 'elo'] = new_home_elo
        team_elo.loc[team_elo.team_long_name == away, 'elo'] = new_away_elo

In [40]:
create_predictions(team_elo, 'expected', 64, 800)

In [41]:
team_elo.sort_values(by='elo', ascending=False).head()

,team_long_name,elo
16,Manchester City,1558.612257
13,Tottenham Hotspur,1368.938948
0,Manchester United,1345.515688
11,Liverpool,1310.865761
9,Chelsea,1236.393027


In [33]:
all_elos = team_elo.copy()

In [42]:
new_elos = team_elo.elo.values[:]
diff_from_mean = new_elos - np.mean(new_elos)
new_elos =  new_elos - (diff_from_mean/3)
all_elos.loc[:, 'newest_elo'] = new_elos
all_elos.sort_values(by='newest_elo', ascending=False).head()

,team_long_name,elo,new_elo,newer_elo,newest_elo
16,Manchester City,1414.307232,1276.204821,1187.167507,1372.408171
13,Tottenham Hotspur,1291.818476,1194.545651,1137.255403,1245.959298
0,Manchester United,1268.952642,1179.301761,1125.171512,1230.343792
11,Liverpool,1250.435221,1166.956814,1117.367932,1207.243840
9,Chelsea,1219.632445,1146.421630,1113.701441,1157.595351


In [24]:
expected_result(1119.53, team_elo[team_elo.team_long_name == 'Leicester City'].new_elo, elo_width)

30    0.674105
Name: new_elo, dtype: float64

In [25]:
with_team_names.loc[:, 'resulty'] = with_team_names.apply(lambda x: calculate_result(x), axis=1)
with_team_names.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,id_away,team_api_id_away,team_fifa_api_id_away,team_long_name_away,team_short_name_away,home_possession,total_prob,pct_win_prob,expected,resulty
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,3458,10261,13.0,Newcastle United,NEW,50,1.047921,0.739744,0.500000,0.5
1,2494,1729,1729,2010/2011,1,2010-08-16 00:00:00,839802,10260,10261,3,...,3458,10261,13.0,Newcastle United,NEW,50,1.048485,0.763006,0.681541,1.0
2,2913,1729,1729,2011/2012,13,2011-11-26 00:00:00,1024845,10260,10261,1,...,3458,10261,13.0,Newcastle United,NEW,50,1.054155,0.677591,0.644366,0.5
3,3353,1729,1729,2012/2013,19,2012-12-26 00:00:00,1229300,10260,10261,4,...,3458,10261,13.0,Newcastle United,NEW,50,1.050441,0.780312,0.710491,1.0
4,3693,1729,1729,2013/2014,15,2013-12-07 00:00:00,1474518,10260,10261,0,...,3458,10261,13.0,Newcastle United,NEW,50,1.027399,0.636165,0.640074,0.0


In [43]:
with_team_names.loc[:, 'label'] = with_team_names.apply(lambda x: get_match_label(x), axis=1)
with_team_names.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,id_away,team_api_id_away,team_fifa_api_id_away,team_long_name_away,team_short_name_away,home_possession,total_prob,pct_win_prob,expected,label
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,3458,10261,13.0,Newcastle United,NEW,56,1.047921,0.739744,0.500000,Draw
1,2494,1729,1729,2010/2011,1,2010-08-16 00:00:00,839802,10260,10261,3,...,3458,10261,13.0,Newcastle United,NEW,64,1.048485,0.763006,0.744386,Win
2,2913,1729,1729,2011/2012,13,2011-11-26 00:00:00,1024845,10260,10261,1,...,3458,10261,13.0,Newcastle United,NEW,70,1.054155,0.677591,0.648628,Draw
3,3353,1729,1729,2012/2013,19,2012-12-26 00:00:00,1229300,10260,10261,4,...,3458,10261,13.0,Newcastle United,NEW,57,1.050441,0.780312,0.779633,Win
4,3693,1729,1729,2013/2014,15,2013-12-07 00:00:00,1474518,10260,10261,0,...,3458,10261,13.0,Newcastle United,NEW,48,1.027399,0.636165,0.629053,Defeat


In [44]:
def get_home_recent_score(match, all_matches):
    last_matches = get_last_matches(
        all_matches,
        match.date,
        match.home_team_api_id,
        x = 10
    )
    score = calculate_team_score(last_matches, match)
    return score

def get_away_recent_score(match, all_matches):
    last_matches = get_last_matches(
        all_matches,
        match.date,
        match.away_team_api_id,
        x = 10
    )
    score = calculate_away_team_score(last_matches, match)
    return score

def get_team_score(match, all_matches):
    last_matches = get_last_matches_against_eachother(
        all_matches,
        match.date,
        match.home_team_api_id,
        match.away_team_api_id,
        x = 25
    )
    score = calculate_team_score(last_matches, match)
    return score

def calculate_team_score(matches, this_match):
    home_matches = matches[matches.home_team_api_id == this_match.home_team_api_id]
    away_matches = matches[matches.away_team_api_id == this_match.home_team_api_id]
    home_offensive_score, home_defensive_score = get_home_score(home_matches)
    away_offensive_score, away_defensive_score = get_away_score(away_matches)
    return (home_offensive_score, home_defensive_score, away_offensive_score, away_defensive_score)

def calculate_away_team_score(matches, this_match):
    home_matches = matches[matches.home_team_api_id == this_match.away_team_api_id]
    away_matches = matches[matches.away_team_api_id == this_match.away_team_api_id]
    home_offensive_score, home_defensive_score = get_home_score(home_matches)
    away_offensive_score, away_defensive_score = get_away_score(away_matches)
    return (home_offensive_score, home_defensive_score, away_offensive_score, away_defensive_score)

def get_home_score(matches):
    total_offensive_score = 0
    total_defensive_score = 0
    for index, match in enumerate(matches.itertuples(index=False, name='Pandas')):
        offensive_score = match.home_team_goal / (index + 1)
        defensive_score = match.away_team_goal / (index + 1)
        total_offensive_score += offensive_score
        total_defensive_score -= defensive_score
    return (total_offensive_score, total_defensive_score)

def get_away_score(matches):
    total_offensive_score = 0
    total_defensive_score = 0
    for index, match in enumerate(matches.itertuples(index=False, name='Pandas')):
        offensive_score = match.away_team_goal / (index + 1)
        defensive_score = match.home_team_goal / (index + 1)
        total_offensive_score += offensive_score
        total_defensive_score -= defensive_score
    return (total_offensive_score, total_defensive_score)

In [45]:
good = with_team_names.apply(lambda x: list(get_team_score(x, with_team_names)), axis=1)

In [46]:
match_stats = pd.DataFrame(good.tolist(), columns = ['home_offensive', 'home_defensive', 'away_offensive', 'away_defensive'])
match_stats.head()

,home_offensive,home_defensive,away_offensive,away_defensive
0,0.000000,0.000000,0.0,0.00
1,1.000000,-1.000000,2.0,-1.00
2,3.500000,-0.500000,1.0,-0.50
3,2.833333,-1.333333,3.5,-1.75
4,5.750000,-3.750000,3.5,-1.75


In [47]:
new_frame = pd.concat([with_team_names, match_stats], axis=1)
new_frame.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,team_short_name_away,home_possession,total_prob,pct_win_prob,expected,label,home_offensive,home_defensive,away_offensive,away_defensive
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,NEW,56,1.047921,0.739744,0.500000,Draw,0.000000,0.000000,0.0,0.00
1,2494,1729,1729,2010/2011,1,2010-08-16 00:00:00,839802,10260,10261,3,...,NEW,64,1.048485,0.763006,0.744386,Win,1.000000,-1.000000,2.0,-1.00
2,2913,1729,1729,2011/2012,13,2011-11-26 00:00:00,1024845,10260,10261,1,...,NEW,70,1.054155,0.677591,0.648628,Draw,3.500000,-0.500000,1.0,-0.50
3,3353,1729,1729,2012/2013,19,2012-12-26 00:00:00,1229300,10260,10261,4,...,NEW,57,1.050441,0.780312,0.779633,Win,2.833333,-1.333333,3.5,-1.75
4,3693,1729,1729,2013/2014,15,2013-12-07 00:00:00,1474518,10260,10261,0,...,NEW,48,1.027399,0.636165,0.629053,Defeat,5.750000,-3.750000,3.5,-1.75


In [48]:
home_mebs = with_team_names.apply(lambda x: list(get_home_recent_score(x, with_team_names)), axis=1)

In [49]:
home_match_stats = pd.DataFrame(home_mebs.tolist(), columns = ['home_recent_offensive', 'home_recent_defensive', 'away_recent_offensive', 'away_recent_defensive'])
home_match_stats.head()

,home_recent_offensive,home_recent_defensive,away_recent_offensive,away_recent_defensive
0,0.000000,0.000000,0.000000,0.000000
1,6.933333,-1.416667,2.700000,0.000000
2,4.516667,-3.650000,3.083333,-0.583333
3,5.000000,-1.583333,4.933333,-3.983333
4,1.950000,-2.316667,4.700000,-4.383333


In [50]:
away_mebs = with_team_names.apply(lambda x: list(get_away_recent_score(x, with_team_names)), axis=1)
away_match_stats = pd.DataFrame(away_mebs.tolist(), columns = ['home_recent_offensive_away', 'home_recent_defensive_away', 'away_recent_offensive_away', 'away_recent_defensive_away'])
away_match_stats.head()

,home_recent_offensive_away,home_recent_defensive_away,away_recent_offensive_away,away_recent_defensive_away
0,0.000000,0.000000,0.000000,0.000000
1,1.700000,-2.600000,0.450000,-3.283333
2,4.316667,-2.116667,3.416667,-4.083333
3,3.083333,-2.366667,1.750000,-3.916667
4,4.566667,-2.600000,1.733333,-4.516667


In [51]:
final_frame = pd.concat([new_frame, home_match_stats, away_match_stats], axis=1)
final_frame.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_offensive,away_defensive,home_recent_offensive,home_recent_defensive,away_recent_offensive,away_recent_defensive,home_recent_offensive_away,home_recent_defensive_away,away_recent_offensive_away,away_recent_defensive_away
0,1729,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,...,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2494,1729,1729,2010/2011,1,2010-08-16 00:00:00,839802,10260,10261,3,...,2.0,-1.00,6.933333,-1.416667,2.700000,0.000000,1.700000,-2.600000,0.450000,-3.283333
2,2913,1729,1729,2011/2012,13,2011-11-26 00:00:00,1024845,10260,10261,1,...,1.0,-0.50,4.516667,-3.650000,3.083333,-0.583333,4.316667,-2.116667,3.416667,-4.083333
3,3353,1729,1729,2012/2013,19,2012-12-26 00:00:00,1229300,10260,10261,4,...,3.5,-1.75,5.000000,-1.583333,4.933333,-3.983333,3.083333,-2.366667,1.750000,-3.916667
4,3693,1729,1729,2013/2014,15,2013-12-07 00:00:00,1474518,10260,10261,0,...,3.5,-1.75,1.950000,-2.316667,4.700000,-4.383333,4.566667,-2.600000,1.733333,-4.516667


In [52]:
X = final_frame[['expected', 'home_offensive', 'home_defensive',
       'away_offensive', 'away_defensive', 'home_recent_offensive',
       'home_recent_defensive', 'away_recent_offensive',
       'away_recent_defensive', 'home_recent_offensive_away',
       'home_recent_defensive_away', 'away_recent_offensive_away',
       'away_recent_defensive_away']].values
y = final_frame.label.values

In [53]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
lr.predict(X)

array(['Win', 'Win', 'Win', ..., 'Defeat', 'Defeat', 'Defeat'],
      dtype=object)

In [55]:
lr.predict_proba(X)

array([[0.23663228, 0.24716955, 0.51619817],
       [0.09445542, 0.15902461, 0.74651997],
       [0.14250371, 0.24173944, 0.61575685],
       ...,
       [0.41320047, 0.28413303, 0.3026665 ],
       [0.51193657, 0.27056746, 0.21749597],
       [0.47752034, 0.3141813 , 0.20829837]])

In [56]:
lr.score(X, y)

0.5281578947368422

In [57]:
lr.predict_proba(X)[-5:]

array([[0.38498129, 0.27463293, 0.34038578],
       [0.42212127, 0.27898249, 0.29889624],
       [0.41320047, 0.28413303, 0.3026665 ],
       [0.51193657, 0.27056746, 0.21749597],
       [0.47752034, 0.3141813 , 0.20829837]])

In [46]:
y[-5:]

array(['Defeat', 'Defeat', 'Win', 'Win', 'Win'], dtype=object)

In [47]:
final_frame[['home_team_goal', 'away_team_goal', 'team_long_name', 'team_long_name_away']].tail()

,home_team_goal,away_team_goal,team_long_name,team_long_name_away
3795,1,2,Watford,Manchester United
3796,2,4,Watford,Manchester United
3797,3,1,Watford,Manchester United
3798,1,0,Brighton,Manchester United
3799,2,1,Huddersfield,Manchester United


In [48]:
final_frame.loc[:, 'pct_draw_prob'] = (1./ final_frame.B365D) / final_frame.total_prob
final_frame.loc[:, 'pct_lose_prob'] = (1./ final_frame.B365A) / final_frame.total_prob
final_frame[['pct_win_prob','pct_draw_prob', 'pct_lose_prob']].head()

,pct_win_prob,pct_draw_prob,pct_lose_prob
0,0.739744,0.173504,0.086752
1,0.763006,0.173410,0.063584
2,0.677591,0.210806,0.111603
3,0.780312,0.146459,0.073229
4,0.636165,0.224788,0.139047


In [49]:
lr.predict_proba(X)[-5:]

array([[0.395839  , 0.27394935, 0.33021165],
       [0.42386361, 0.27773879, 0.2983976 ],
       [0.41028991, 0.28350096, 0.30620913],
       [0.5124411 , 0.26908168, 0.21847722],
       [0.48909004, 0.31340839, 0.19750156]])

In [50]:
final_frame[['pct_win_prob', 'pct_draw_prob', 'pct_lose_prob']].tail()

,pct_win_prob,pct_draw_prob,pct_lose_prob
3795,0.225300,0.286926,0.487774
3796,0.185119,0.249839,0.565042
3797,0.163072,0.232959,0.603969
3798,0.204624,0.269990,0.525386
3799,0.088588,0.194893,0.716519


In [52]:
home_mebs = with_team_names.apply(lambda x: list(get_home_recent_score(x, with_team_names)), axis=1)

def get_home_recent_score(match, all_matches):
    last_matches = get_last_matches(
        all_matches,
        match.date,
        match.home_team_api_id,
        x = 10
    )
    score = calculate_team_score(last_matches, match)
    return score

home_team_api = team_data[]

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [61]:
home_team_api = team_data[team_data.team_long_name == 'Manchester United'].team_api_id.values[0]
away_team_api = team_data[team_data.team_long_name == 'Leicester City'].team_api_id.values[0]

last_matches = get_last_matches(with_team_names, str(datetime.date.today()), home_team_api)
score = calculate_team_score(last_matches, with_team_names)
score

ValueError: Can only compare identically-labeled Series objects

In [58]:
with_team_names[with_team_names.home_team_api_id == home_team_api].head()

ValueError: Can only compare identically-labeled Series objects

In [78]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

ohe = OneHotEncoder()
le = LabelEncoder()
better = ohe.fit_transform(le.fit_transform(y).reshape(-1, 1))

In [79]:
from sklearn.metrics import log_loss

log_loss(better.toarray(), lr.predict_proba(X))

0.98268837960003

In [80]:
log_loss(better.toarray(), final_frame[['pct_win_prob', 'pct_draw_prob', 'pct_lose_prob']].values)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [81]:
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=3, test_size=.2, random_state=0)
for train_index, test_index in rs.split(X):
    train_x = X[train_index]
    train_y = y[train_index]
    test_x = X[test_index]
    test_y = y[test_index]
    lr = LogisticRegression()
    lr.fit(train_x, train_y)
    
    ohe = OneHotEncoder()
    le = LabelEncoder()
    better = ohe.fit_transform(le.fit_transform(train_y).reshape(-1, 1))
    
    print("Train loss: {0:.3f}".format(log_loss(better.toarray(), lr.predict_proba(train_x))))
    
    test_better = ohe.transform(le.transform(test_y).reshape(-1, 1))
    
    print("Test loss: {0:.3f}".format(log_loss(test_better.toarray(), lr.predict_proba(test_x))))

Train loss: 0.982
Test loss: 0.993
Train loss: 0.981
Test loss: 0.995
Train loss: 0.986
Test loss: 0.981


In [82]:
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=3, test_size=.2, random_state=0)
for train_index, test_index in rs.split(X):
    train_x = X[train_index]
    train_y = y[train_index]
    test_x = X[test_index]
    test_y = y[test_index]
    lr = LogisticRegression()
    #pf = PolynomialFeatures()
    #new_train_x = pf.fit_transform(train_x)
    #new_test_x = pf.transform(test_x)
    lr.fit(train_x, train_y)
    
    ohe = OneHotEncoder()
    le = LabelEncoder()
    better = ohe.fit_transform(le.fit_transform(train_y).reshape(-1, 1))
    
    print("Train loss: {0:.3f}".format(log_loss(better.toarray(), lr.predict_proba(train_x))))
    
    test_better = ohe.transform(le.transform(test_y).reshape(-1, 1))
    
    print("Test loss: {0:.3f}".format(log_loss(test_better.toarray(), lr.predict_proba(test_x))))

Train loss: 0.982
Test loss: 0.993
Train loss: 0.981
Test loss: 0.995
Train loss: 0.986
Test loss: 0.981


In [237]:
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import ShuffleSplit

from sklearn.ensemble import RandomForestClassifier

rs = ShuffleSplit(n_splits=3, test_size=.1, random_state=382)
for train_index, test_index in rs.split(X):
    train_x = X[train_index]
    train_y = y[train_index]
    test_x = X[test_index]
    test_y = y[test_index]
    rf = RandomForestClassifier(max_depth=3, min_samples_split=3, min_samples_leaf=2)
    pf = PolynomialFeatures()
    new_train_x = pf.fit_transform(train_x)
    new_test_x = pf.transform(test_x)
    rf.fit(new_train_x, train_y)
    
    ohe = OneHotEncoder()
    le = LabelEncoder()
    better = ohe.fit_transform(le.fit_transform(train_y).reshape(-1, 1))
    
    print("Train loss: {0:.3f}".format(log_loss(better.toarray(), rf.predict_proba(new_train_x))))
    
    test_better = ohe.transform(le.transform(test_y).reshape(-1, 1))
    
    print("Test loss: {0:.3f}".format(log_loss(test_better.toarray(), rf.predict_proba(new_test_x))))

Train loss: 0.975
Test loss: 1.021
Train loss: 0.971
Test loss: 1.027
Train loss: 0.976
Test loss: 0.998


In [83]:
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import ShuffleSplit

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

rs = ShuffleSplit(n_splits=3, test_size=.1, random_state=18)
for train_index, test_index in rs.split(X):
    train_x = X[train_index]
    train_y = y[train_index]
    test_x = X[test_index]
    test_y = y[test_index]
    gb = GradientBoostingClassifier(subsample=0.65, min_samples_leaf=3, min_samples_split=4)
    #pf = PolynomialFeatures(interaction_only=True, include_bias=False)
    #new_train_x = pf.fit_transform(train_x)
    #new_test_x = pf.transform(test_x)
    gb.fit(train_x, train_y)
    
    ohe = OneHotEncoder()
    le = LabelEncoder()
    better = ohe.fit_transform(le.fit_transform(train_y).reshape(-1, 1))
    
    print("Train loss: {0:.3f}".format(log_loss(better.toarray(), gb.predict_proba(train_x))))
    
    test_better = ohe.transform(le.transform(test_y).reshape(-1, 1))
    
    print("Test loss: {0:.3f}".format(log_loss(test_better.toarray(), gb.predict_proba(test_x))))

Train loss: 0.798
Test loss: 0.993
Train loss: 0.797
Test loss: 1.020
Train loss: 0.798
Test loss: 1.030


In [84]:
for label, coef in zip(['bias', 'expected', 'other_expected',
       'final_expected', 'home_offensive', 'home_defensive',
       'away_offensive', 'away_defensive', 'home_recent_offensive',
       'home_recent_defensive', 'away_recent_offensive',
       'away_recent_defensive', 'home_recent_offensive_away',
       'home_recent_defensive_away', 'away_recent_offensive_away',
       'away_recent_defensive_away'], lr.coef_[0]):
    print("{0} : {1:.2f}".format(label, coef))

bias : -3.90
expected : -0.33
other_expected : -0.03
final_expected : -0.04
home_offensive : -0.06
home_defensive : -0.07
away_offensive : 0.01
away_defensive : -0.00
home_recent_offensive : -0.07
home_recent_defensive : -0.04
away_recent_offensive : 0.01
away_recent_defensive : 0.02
home_recent_offensive_away : 0.03
home_recent_defensive_away : 0.02


In [85]:
lr.predict_proba(X)[:5]

array([[0.20395042, 0.2417693 , 0.55428028],
       [0.09978947, 0.17537299, 0.72483754],
       [0.12718423, 0.25496784, 0.61784794],
       [0.08734989, 0.19465913, 0.71799098],
       [0.12898628, 0.2506866 , 0.62032712]])

In [86]:
final_frame[['pct_win_prob', 'pct_draw_prob', 'pct_lose_prob']].values[:5]

array([[0.73974445, 0.1735037 , 0.08675185],
       [0.76300578, 0.1734104 , 0.06358382],
       [0.67759079, 0.21080602, 0.11160319],
       [0.78031212, 0.14645858, 0.07322929],
       [0.63616463, 0.22478796, 0.13904741]])

In [87]:
draw_predictions = final_frame[(lr.predict_proba(X)[:, 1] - final_frame.pct_draw_prob ) <= 0.]

In [88]:
BET = 1
RUNNING_TOTAL = 0
CORRECT_COUNT = 0

for index, row in draw_predictions.sort_values(by='date').iterrows():
    odds = row.B365D
    result = row.label
    if result == 'Draw':
        CORRECT_COUNT += 1
        RUNNING_TOTAL += odds*BET
    else:
        RUNNING_TOTAL -= BET

In [89]:
print("Money made: {0:.2f}".format(RUNNING_TOTAL))
print("Correct count: {0}".format(CORRECT_COUNT))

Money made: 506.95
Correct count: 558


In [90]:
final_frame.sort_values(by='B365D').head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,home_recent_offensive,home_recent_defensive,away_recent_offensive,away_recent_defensive,home_recent_offensive_away,home_recent_defensive_away,away_recent_offensive_away,away_recent_defensive_away,pct_draw_prob,pct_lose_prob
3600,55303,1729,51959,2017/2018,23,2018-01-15 00:00:00,2522972,8659,10204,2,...,1.733333,-2.066667,1.700000,-3.950000,2.750000,-3.250000,0.000000,-2.366667,0.323132,0.223879
2833,55092,1729,51959,2017/2018,27,2018-02-12 00:00:00,2523010,10003,8191,1,...,3.750000,-2.716667,2.366667,-3.683333,1.533333,-2.916667,1.500000,-2.000000,0.313588,0.289466
2848,55087,1729,51959,2017/2018,18,2017-12-18 00:00:00,2522915,10204,8191,0,...,2.116667,-6.116667,0.850000,-3.333333,2.366667,-0.500000,1.333333,-2.250000,0.324408,0.304132
3687,2049,1729,1729,2008/2009,4,2008-09-13 00:00:00,489072,8650,10260,2,...,2.000000,-1.000000,0.500000,0.000000,1.000000,-1.000000,1.000000,0.000000,0.317073,0.365854
321,55211,1729,51959,2017/2018,26,2018-02-05 00:00:00,2522995,10204,8654,3,...,1.333333,-5.000000,1.000000,-3.233333,2.833333,-2.583333,6.250000,-2.500000,0.323896,0.262619


In [91]:
with_team_names[['team_long_name', 'team_long_name_away', 'season', 'home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A']].sort_values(by='B365D').head()

,team_long_name,team_long_name_away,season,home_team_goal,away_team_goal,B365H,B365D,B365A
3600,West Bromwich Albion,Brighton,2017/2018,2,0,2.14,3.0,4.33
2833,Swansea City,Burnley,2017/2018,1,0,2.37,3.0,3.25
2848,Brighton,Burnley,2017/2018,0,0,2.62,3.0,3.20
3687,Liverpool,Manchester United,2008/2009,2,1,3.00,3.0,2.60
321,Brighton,West Ham United,2017/2018,3,1,2.35,3.0,3.70


In [93]:
with_team_names[['team_long_name', 'team_long_name_away', 'season', 'home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A']].sort_values(by='B365D', ascending=False).head()

,team_long_name,team_long_name_away,season,home_team_goal,away_team_goal,B365H,B365D,B365A
3582,Manchester City,Huddersfield,2017/2018,0,0,1.08,13.0,34.0
625,Arsenal,Sunderland,2016/2017,2,0,1.09,13.0,29.0
2826,Manchester City,Burnley,2017/2018,3,0,1.11,12.0,26.0
269,Manchester City,West Ham United,2017/2018,2,1,1.08,11.0,21.0
3282,Manchester City,Cardiff City,2013/2014,4,2,1.13,11.0,23.0


In [94]:
final_frame.groupby('label').size() / final_frame.shape[0]

label
Defeat    0.285263
Draw      0.254211
Win       0.460526
dtype: float64

In [95]:
final_frame.pct_draw_prob.describe()

count    3790.000000
mean        0.251396
std         0.047596
min         0.074519
25%         0.229361
50%         0.270030
75%         0.287348
max         0.325175
Name: pct_draw_prob, dtype: float64

In [96]:
final_frame.loc[:, 'total_odds'] = final_frame[['B365H', 'B365D', 'B365A']].sum(axis=1)
final_frame.sort_values(by='total_odds', ascending=True).head()[['team_long_name', 'team_long_name_away', 'season', 'home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A']]

,team_long_name,team_long_name_away,season,home_team_goal,away_team_goal,B365H,B365D,B365A
2106,Everton,Manchester City,2017/2018,1,3,NaN,NaN,NaN
3156,West Ham United,Southampton,2017/2018,3,0,NaN,NaN,NaN
1581,Crystal Palace,Liverpool,2017/2018,1,2,NaN,NaN,NaN
3045,Crystal Palace,Swansea City,2016/2017,1,2,NaN,NaN,NaN
3044,Crystal Palace,Swansea City,2017/2018,0,2,NaN,NaN,NaN


In [97]:
final_frame.groupby('season').size()

season
2008/2009    380
2009/2010    380
2010/2011    380
2011/2012    380
2012/2013    380
2013/2014    380
2014/2015    380
2015/2016    380
2016/2017    380
2017/2018    380
dtype: int64